In [1]:
import requests
import urllib.parse
from bs4 import BeautifulSoup
import mpl_finance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from datetime import datetime
from openpyxl import load_workbook

#繪圖
import plotly.graph_objects as go
from plotly import subplots

%run _CrawBase.ipynb
%run _BaseInfo.ipynb

df_info=_baseInfo.copy()

C:\Users\user\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [2]:
# 計算 RSI 指標
def calculate_rsi(data, window=14):
    delta = data['收盤價'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [3]:
def get_interval(value_to_check, data):
    # 转换数据为浮点数并去除负值和零值
    #data = [float(num) for num in data if float(num) > 0]
    cleaned_numbers = []
    for num in data:
        try:
            value = float(num)
            if value > 0:
                cleaned_numbers.append(value)
        except ValueError:
            # 忽略无法转换的字符串
            continue
    data= cleaned_numbers  
    #print(value_to_check)
    value_to_check=float(value_to_check)
    
    # 计算均值和标准差
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)  # 使用 ddof=1 计算样本标准差

    confidence_levels = [0.40,0.50,0.65, 0.90, 0.95, 0.99]
    
    best_interval = None
    for level in confidence_levels:
        z_score = stats.norm.ppf(1 - (1 - level) / 2)  # 获取 z 分数
        margin_of_error = z_score * std_dev
        
        #print(mean)
        #print(margin_of_error)
        lower_bound = mean - margin_of_error
        upper_bound = mean + margin_of_error
        # 检查 value_to_check 是否在当前置信区间内
        if lower_bound <= value_to_check <= upper_bound:
            best_interval = (level, lower_bound, upper_bound)
            break  # 如果找到包含 value_to_check 的置信区间，直接退出
    if best_interval:
        level, lower_bound, upper_bound = best_interval
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"

        #print(f"数据点 {value_to_check} 落在置信水平 {level * 100}% 的{interval_type}: [{lower_bound:.2f}, {upper_bound:.2f}]")
        return (level,interval_type,lower_bound, upper_bound)
    else:
        if mean <value_to_check:
            interval_type = "正區間"
        else:
            interval_type = "負區間"
        
        return (1,interval_type,value_to_check, value_to_check)
        return None

In [4]:
def data_process(RowData_df_craw_stock):

    # 讀取並處理基本資料格式
    df = RowData_df_craw_stock.copy()
    df =  df.drop_duplicates()
    
    # 分割日期和年份，將年份轉換為民國年，並重新組合日期
    df['Year'] = df['日期'].str.split('/', 1).str[0].astype(int) + 1911
    df['Date'] = df['日期'].str.split('/', 1).str[1]
    df['日期'] = df.apply(lambda x: f"{x['Year']}/{x['Date']}", axis=1)
    df['Date_'] = df['日期']
    df['日期'] = df['日期'].str.replace("＊", "")
   

    # 複製數據
    stock_data = df.copy()

    # 轉換數據格式
    stock_data['成交金額'] = stock_data['成交金額'].str.replace(',', '').astype(float)
    stock_data['收盤價'] = stock_data['收盤價'].replace('--', np.nan).str.replace(',', '').astype(float)
    stock_data['開盤價'] = stock_data['開盤價'].replace('--', np.nan).str.replace(',', '').astype(float)
    stock_data['成交股數'] = stock_data['成交股數'].str.replace(',', '').astype(float)
    stock_data['年月日']=stock_data['日期']

    
    # 計算 EMA (指數移動平均線)
    # 使用 12 日的收盤價計算快速 EMA
    stock_data['EMA_12'] = stock_data['收盤價'].ewm(span=12, adjust=False).mean()
    # 使用 26 日的收盤價計算慢速 EMA
    stock_data['EMA_26'] = stock_data['收盤價'].ewm(span=26, adjust=False).mean()

    # 計算 MACD (移動平均收斂散度)
    # MACD 線 = 快速 EMA - 慢速 EMA
    stock_data['MACD'] = stock_data['EMA_12'] - stock_data['EMA_26']
    stock_data['MACD-SL'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()


    # 計算 Signal 線 (信號線)
    # 使用 9 日的 MACD 值計算信號線
    stock_data['Signal_Line'] = stock_data['MACD'].ewm(span=9, adjust=False).mean()

    # 計算 DIF (差離值)
    # DIF = MACD 線 - 信號線
    stock_data['DIF']= stock_data['MACD'] - stock_data['MACD-SL']

    
    
    # 計算 MACD 黃金交叉
    # 當 MACD 線由下而上穿過信號線時，標記為黃金交叉
    stock_data['MACD_golden_cross'] = (stock_data['MACD'] > stock_data['Signal_Line']) & (stock_data['MACD'].shift(1) <= stock_data['Signal_Line'].shift(1))

    # 計算 KD 指標
    # 計算 %K 線和 %D 線
    # 計算過去 9 天的最低價
    stock_data['Lowest_Low'] = stock_data['收盤價'].rolling(window=9).min()
    # 計算過去 9 天的最高價
    stock_data['Highest_High'] = stock_data['收盤價'].rolling(window=9).max()
    # 計算 %K 線
    stock_data['%K'] = (stock_data['收盤價'] - stock_data['Lowest_Low']) / (stock_data['Highest_High'] - stock_data['Lowest_Low']) * 100
    # 計算 %D 線，為 %K 線的 3 日移動平均
    stock_data['%D'] = stock_data['%K'].rolling(window=3).mean()

    # 計算 KD 黃金交叉
    # 當 %K 線由下而上穿過 %D 線時，標記為黃金交叉
    stock_data['KD_golden_cross'] = (stock_data['%K'] > stock_data['%D']) & (stock_data['%K'].shift(1) <= stock_data['%D'].shift(1))


    # 計算成交量相關指標
    # 計算短期成交量移動平均線（20天）
    stock_data['Volume_MA_short'] = stock_data['成交金額'].rolling(window=20).mean()
    # 計算長期成交量移動平均線（50天）
    stock_data['Volume_MA_long'] = stock_data['成交金額'].rolling(window=50).mean()
    # 計算成交量震盪指標
    stock_data['Volume_Oscillator'] = ((stock_data['Volume_MA_short'] - stock_data['Volume_MA_long']) / stock_data['Volume_MA_long']) * 100
    # 計算成交量突破
    # 當成交量大於20天成交量移動平均值時，標記為成交量突破
    stock_data['Volume_breakout'] = stock_data['成交金額'] > stock_data['成交金額'].rolling(window=20).mean()
    stock_data['Volume_breakout'] = (stock_data['成交金額'] > stock_data['成交金額'].rolling(window=20).mean()) & (stock_data['收盤價'] > stock_data['開盤價'])
    # 計算移動平均線
    # 計算短期移動平均線（20天）
    stock_data['MA_short'] = stock_data['收盤價'].rolling(window=5).mean()
    # 計算長期移動平均線（50天）
    stock_data['MA_long'] = stock_data['收盤價'].rolling(window=20).mean()
    
    
    stock_data['MA_break']= (stock_data['MA_short'] > stock_data['MA_long'] 
                            ) & (
                             stock_data['MA_short'].shift(3) <= stock_data['MA_long'].shift(3)
                            )

    

    # 計算均線的波動率
    # 計算短期移動平均線的波動率（10天標準差）
    stock_data['MA_short_volatility'] = stock_data['MA_short'].rolling(window=10).std()
    # 計算長期移動平均線的波動率（10天標準差）
    stock_data['MA_long_volatility'] = stock_data['MA_long'].rolling(window=10).std()
    # 當短期移動平均線的波動率大於長期移動平均線的波動率時，標記為糾纏狀態
    stock_data['Is_Entangled'] = stock_data['MA_short_volatility'] > stock_data['MA_long_volatility']

    # 計算均線的變化率
    # 計算短期移動平均線的變化率
    stock_data['MA_short_change'] = stock_data['MA_short'].diff()
    # 計算長期移動平均線的變化率
    stock_data['MA_long_change'] = stock_data['MA_long'].diff()



    

    # 計算布林帶
    window = 20  # 計算移動平均線和標準差的窗口大小
    num_std_dev = 2  # 標準差倍數，通常設置為2
    # 計算布林帶的中線（移動平均線）
    stock_data['Bollinger_MA'] = stock_data['收盤價'].rolling(window=window).mean()
    # 計算布林帶的標準差
    stock_data['Bollinger_STD'] = stock_data['收盤價'].rolling(window=window).std()
    # 計算布林帶的上限
    stock_data['Bollinger_upper'] = stock_data['Bollinger_MA'] + (num_std_dev * stock_data['Bollinger_STD'])
    # 計算布林帶的下限
    stock_data['Bollinger_lower'] = stock_data['Bollinger_MA'] - (num_std_dev * stock_data['Bollinger_STD'])
    # 當收盤價高於布林帶上限時，標記為布林帶突破
    stock_data['Bollinger_breakout'] = stock_data['收盤價'] > stock_data['Bollinger_upper']

    
    # 計算 RSI
    stock_data['RSI'] = calculate_rsi(stock_data)
    # 當 RSI 從30以下反彈到30以上時，標記為 RSI 反彈
    stock_data['RSI_rebound'] = (stock_data['RSI'] > 30) & (stock_data['RSI'].shift(1) <= 30)

    # 計算漲跌價差和顏色標記
    # 計算收盤價的變化量
    stock_data['Price_Change'] = stock_data['收盤價'].diff()
    
    stock_data['Volume_Price_Change'] = np.where(stock_data['Price_Change'] > 0, 1, -1) * stock_data['成交金額']
    stock_data['Volume_Price_Change_sum'] = stock_data['Volume_Price_Change'].rolling(window=3).sum()
    
    # 根據價差標記顏色，漲為紅色，跌為綠色
    stock_data['Bar_Color'] = stock_data['Price_Change'].apply(lambda x: 'red' if x > 0 else 'green')

    # 計算連續多日收盤價高於移動平均線
    # 當收盤價高於短期移動平均線時，標記為 True
    stock_data['Close_above_MA'] = stock_data['收盤價'] > stock_data['MA_short']
    # 計算連續 5 日收盤價都高於短期移動平均線的天數
    stock_data['Close_above_MA_5days'] = stock_data['Close_above_MA'].rolling(window=5).sum() == 5

    # 計算 Signal_Balance 指標
    # 應用自定義函數 get_analysis_and_count 計算 Analysis 和 Signal_Balance
    stock_data[['Analysis', 'Signal_Balance']] = stock_data.apply(get_analysis_and_count, axis=1, result_type='expand')
    
    
    # 計算 Signal_Balance 的 10 日移動平均線
    stock_data['Signal_Balance_roll'] = stock_data['Signal_Balance'].rolling(window=10).mean()
    # 計算 Signal_Balance 10 日均線的變化量
    stock_data['Signal_Balance_roll_diff'] = stock_data['Signal_Balance_roll'].diff()
    # 當 Signal_Balance 10 日均線上升時，標記為處於上升趨勢
    stock_data['Signal_Balance_uptrend'] = stock_data['Signal_Balance_roll_diff'] > 0
    # 計算連續三日 Signal_Balance 10 日均線上升的天數
    stock_data['Signal_Balance_uptrend_3days'] = stock_data['Signal_Balance_uptrend'].rolling(window=3).sum() == 3

    # 應用趨勢判斷到每一筆資料
    # 使用自定義函數 determine_trend 判斷趨勢
    stock_data['Trend'] = stock_data.apply(determine_trend, axis=1)
    # 將趨勢結果加入到 Analysis 中
    stock_data['Analysis'] += stock_data['Trend']

    
    
    
    
    
    # 定義每個指標的權重
    # 我希望在0~1震盪 所以權重加總1
    weights = {
        'Signal_Balance_uptrend_3days': 0.15,  # Signal_Balance 連續三日上升的權重
        'MACD_golden_cross': 0.15,             # MACD 黃金交叉的權重
        'KD_golden_cross': 0.1,               # KD 黃金交叉的權重
        'Volume_breakout': 0.15,               # 成交量突破的權重
        'RSI_rebound':0.15,                   # RSI 反彈的權重
        'Bollinger_breakout': 0.15,            # 布林帶突破的權重
        'Close_above_MA_5days': 0.15           # 連續五日收盤價高於移動平均線的權重
    }
    threshold = 0.25  # 設置信號的閾值

    # 計算每個條件的加權信號
    stock_data['Weighted_Signal'] = (
        (stock_data['Signal_Balance_uptrend_3days'] * weights['Signal_Balance_uptrend_3days']) +
        (stock_data['MACD_golden_cross'] * weights['MACD_golden_cross']) +
        (stock_data['KD_golden_cross'] * weights['KD_golden_cross']) +
        (stock_data['Volume_breakout'] * weights['Volume_breakout']) +
        (stock_data['RSI_rebound'] * weights['RSI_rebound']) +
        (stock_data['Bollinger_breakout'] * weights['Bollinger_breakout']) +
        (stock_data['Close_above_MA_5days'] * weights['Close_above_MA_5days'])
    )
    
    # 當加權信號大於閾值時，標記為 True
    stock_data['Weighted_Signa_over_threshold'] = stock_data['Weighted_Signal']/2 > threshold
    
    stock_data['Weighted_Signal'] = stock_data['Weighted_Signal']
    stock_data['Weighted_Signal_roll'] = stock_data['Weighted_Signal'].rolling(window=6).mean()
    stock_data['Weighted_Signal_roll_'] = stock_data['Weighted_Signal'].rolling(window=15).mean()
    

    stock_data['Weighted_Signal_golden_cross'] = (stock_data['Weighted_Signal_roll'] > stock_data['Weighted_Signal_roll_'] 
                                                 ) & (
                                                stock_data['Weighted_Signal_roll'].shift(1) <= stock_data['Weighted_Signal_roll_'].shift(1)
                                                 )

    
    return stock_data

In [5]:
# 判斷每一筆資料的股價趨勢
def determine_trend(row):
    if row['MA_short'] > row['MA_long']:
        if row['MA_short_change'] > 0:
            return '均線:上漲趨勢'
        else:
            return '均線:橫盤震盪'
    elif row['MA_short'] < row['MA_long']:
        if row['MA_short_change'] < 0:
            return '均線:下降趨勢'
        else:
            return '均線:橫盤震盪'
    else:
        return '均線:無明顯趨勢'

### `Weighted_Signal` 指標邏輯說明

**目的**：
`Weighted_Signal` 指標旨在綜合多個技術指標的信號，通過為每個指標分配權重來計算最終的加權信號。這個加權信號可以幫助投資者更好地判斷市場的買入或賣出時機。

### 計算過程

1. **定義每個指標的權重**：
   - **`Signal_Balance_uptrend_3days`**: 連續三日 `Signal_Balance` 上升的權重為 1.0。
   - **`MACD_golden_cross`**: MACD 黃金交叉的權重為 1.5。
   - **`KD_golden_cross`**: KD 黃金交叉的權重為 1.2。
   - **`Volume_breakout`**: 成交量突破的權重為 1.0。
   - **`RSI_rebound`**: RSI 反彈的權重為 1.1。
   - **`Bollinger_breakout`**: 布林帶突破的權重為 1.3。
   - **`Close_above_MA_5days`**: 連續五日收盤價高於移動平均線的權重為 1.0。

2. **計算每個條件的加權信號**：
   - 對每個技術指標的信號值進行加權，並將結果加總。公式為：
     ```python
     stock_data['Weighted_Signal'] = (
         (stock_data['Signal_Balance_uptrend_3days'] * weights['Signal_Balance_uptrend_3days']) +
         (stock_data['MACD_golden_cross'] * weights['MACD_golden_cross']) +
         (stock_data['KD_golden_cross'] * weights['KD_golden_cross']) +
         (stock_data['Volume_breakout'] * weights['Volume_breakout']) +
         (stock_data['RSI_rebound'] * weights['RSI_rebound']) +
         (stock_data['Bollinger_breakout'] * weights['Bollinger_breakout']) +
         (stock_data['Close_above_MA_5days'] * weights['Close_above_MA_5days'])
     )
     ```

3. **設置閾值**：
   - `threshold` 設定為 2.5，用於判斷加權信號是否超過這個閾值。

### 使用方法

- **加權信號**：
  - `Weighted_Signal` 是各技術指標加權後的綜合信號值。
  - 計算後的 `Weighted_Signal` 值如果超過設置的閾值（2.5），則表明市場可能存在較強的買入信號。

### 總結

`Weighted_Signal` 指標通過加權計算來綜合多個技術指標的信號，這樣可以有效地將不同指標的影響納入考量，提供一個更全面的市場信號評估。加權的方式允許根據不同指標的重要性調整其影響力，而閾值則用於識別強信號的條件。這個指標可以幫助投資者更精確地把握市場的買賣時機。

**`Signal_Balance` 指標邏輯整理**

**目的**：
`Signal_Balance` 是一個綜合技術指標，用於評估市場的買賣信號。它通過分析多種技術指標（如股價與移動平均線、成交量與成交量震盪指標、KD 指標和 MACD 指標）來生成一個綜合的信號平衡值。這個值可以幫助判斷市場當前的買賣傾向。

### 計算過程

1. **股價與移動平均線**：
   - **買入信號**：
     - 股價高於短期均線（MA_short），且短期均線高於長期均線（MA_long）。
   - **賣出信號**：
     - 股價低於短期均線，且短期均線低於長期均線。
   - **說明**：這些條件表明市場的短期走勢與長期走勢的關係，從而推測可能的買入或賣出時機。

2. **成交金額與成交量震盪指標**：
   - **買入信號**：
     - 成交金額高於短期均線（Volume_MA_short），成交量震盪指標（Volume_Oscillator）為正，且價格變化（Price_Change）為正。
   - **賣出信號**：
     - 成交金額低於短期均線，成交量震盪指標為負，且價格變化為負。
   - **說明**：這些條件考察成交量的變化以及價格走勢，幫助識別市場的強勢或弱勢信號。

3. **KD 指標**：
   - **買入信號**：
     - %K 指標高於 %D 指標，且兩個指標都低於 50。
   - **賣出信號**：
     - %K 指標低於 %D 指標，且兩個指標都高於 80。
   - **說明**：KD 指標用於衡量市場的超買或超賣狀態，買入和賣出信號的生成基於這些狀態的變化。

4. **MACD 指標**：
   - **買入信號**：
     - MACD 線高於信號線（Signal_Line）。
   - **賣出信號**：
     - MACD 線低於信號線。
   - **說明**：MACD 指標用於跟蹤市場趨勢的變化，買入或賣出信號基於 MACD 線與信號線的相對位置。

### 綜合信號平衡值（`Signal_Balance`）

- **計算方法**：
  - 通過對以上各項指標的分析，計算買入信號和賣出信號的數量。
  - 綜合信號平衡值（`Signal_Balance`）= 買入信號的數量 - 賣出信號的數量。
- **解釋**：
  - 如果綜合信號平衡值為正，則表明買入信號佔優，市場傾向於買入。
  - 如果綜合信號平衡值為負，則表明賣出信號佔優，市場傾向於賣出。

### 總結

`Signal_Balance` 是一個通過技術指標綜合計算得出的信號平衡值，用於評估市場的買賣傾向。它結合了股價與均線關係、成交量變化、KD 指標和 MACD 指標，提供了一個直觀的市場買賣信號判斷依據。通過分析這些信號，投資者可以更好地做出交易決策。

當然，這兩個指標各有其優缺點。以下是對 `Signal_Balance_uptrend_3days` 和 `Weighted_Signal` 指標的優缺點分析：

### `Signal_Balance_uptrend_3days`

**優點**：
1. **簡單明瞭**：
   - 易於理解和計算，只需關注 `Signal_Balance` 指標在三日內的變化趨勢。
   
2. **趨勢跟蹤**：
   - 能夠有效捕捉到短期內的上升趨勢，適合用來識別持續的市場趨勢。

3. **避免噪音**：
   - 短期內持續上升的趨勢可能代表著更強的市場力量，能夠幫助過濾掉短期波動帶來的噪音。

**缺點**：
1. **延遲性**：
   - 由於是基於過去三天的數據，可能存在一定的延遲，無法即時反映市場的快速變化。

2. **單一指標依賴**：
   - 只考慮了 `Signal_Balance` 指標的一部分信息，可能忽略了其他重要的市場信號。

3. **市場狀況影響**：
   - 在震盪市場中，持續上升的信號可能不夠穩定，容易受到短期波動的影響。

### `Weighted_Signal`

**優點**：
1. **綜合分析**：
   - 通過綜合多個技術指標，提供了一個全面的市場信號，能夠更好地反映市場狀況。

2. **靈活性**：
   - 允許根據不同指標的重要性分配不同的權重，靈活調整策略以適應不同的市場環境。

3. **信號強度**：
   - 可以設置閾值來篩選強信號，使得交易決策更加明確。

**缺點**：
1. **複雜性**：
   - 計算過程較為複雜，需要對每個指標進行加權和綜合，可能對新手投資者不太友好。

2. **權重選擇**：
   - 權重的設置是主觀的，可能會受到分析師偏好的影響，這可能導致某些指標被過度或不足重視。

3. **市場變化適應性**：
   - 隨著市場環境的變化，固定的權重和閾值可能需要調整，以保持指標的有效性。

### 總結

- **`Signal_Balance_uptrend_3days`** 適合需要簡單趨勢判斷的場景，能夠有效識別短期趨勢，但在震盪市場中可能表現不佳。
- **`Weighted_Signal`** 提供了更為全面的市場分析，能夠整合多種信號來制定交易決策，但計算複雜且依賴於主觀設定的權重。

根據不同的需求，您可以選擇使用單一指標來簡單快速地獲取市場信號，或使用加權指標來進行更為全面的分析。

In [6]:
def get_analysis_and_count(row):
    
    # 我希望在0~1震盪 所以count加總1
    x=0.25
    
    #计算综合信号平衡值：通过计算买入信号和卖出信号的数量差，得到一个综合的信号值，用于评估当前市场的买卖倾向。
    buy_count = 0
    sell_count = 0
    analysis = ""

    # 股價與移動平均線
    if row['收盤價'] > row['MA_short'] and row['MA_short'] > row['MA_long']:
        analysis += "<span style='color:red;'>[股價高於短期均線，短期均線高於長期均線]：買入信號</span><br>"
        buy_count += x
    elif row['收盤價'] < row['MA_short'] and row['MA_short'] < row['MA_long']:
        analysis += "<span style='color:green;'>[股價低於短期均線，短期均線低於長期均線]：賣出信號</span><br>"
        sell_count += x
    
    # 成交金額與成交量震盪指標
    if row['成交金額'] > row['Volume_MA_short'] and row['Volume_Oscillator'] > 0 and row['Price_Change'] >0:
        analysis += "<span style='color:red;'>[成交量大於均線，成交量震盪指標為正]：買入信號</span><br>"
        buy_count += x
    elif row['成交金額'] < row['Volume_MA_short'] and row['Volume_Oscillator'] < 0 and row['Price_Change'] <0:
        analysis += "<span style='color:green;'>[成交量低於均線，成交量震盪指標為負]：賣出信號</span><br>"
        sell_count += x
    
    # KD 指標
    if row['%K'] > row['%D'] and row['%K']<50 and row['%D'] <50 :
        analysis += "<span style='color:red;'>[KD 指標 %K 高於 %D]：買入信號</span><br>"
        buy_count += x
    elif row['%K'] < row['%D'] and row['%K']>80 and row['%D'] >80 :
        analysis += "<span style='color:green;'>[KD 指標 %K 低於 %D]：賣出信號</span><br>"
        sell_count += x
        
        
    #  MACD  
    # 定義成交金額的輔助條件
    volume_condition = row['成交金額'] > row['Volume_MA_short']

    # 結合 RSI 指標進行強買入信號的判斷
    rsi_condition = row['RSI'] < 30  # RSI 指標低於30，表示超賣狀態，可能存在反彈機會
    if row['MACD'] > row['MACD-SL'] and volume_condition and rsi_condition:
        analysis += "<span style='color:red;'>[MACD 高於信號線 (MACD-SL)，成交量增加，RSI 超賣]：強買入信號</span><br>"
        buy_count += x

    # 結合 KD 指標進行強賣出信號的判斷
    kd_condition = row['%K'] > row['%D']  # KD 指標中 %K 線高於 %D 線，表明短期動能強於長期動能
    if row['MACD'] < row['MACD-SL'] and volume_condition and kd_condition:
        analysis += "<span style='color:green;'>[MACD 低於信號線 (MACD-SL)，成交量增加，KD 指標]：強賣出信號</span><br>"
        sell_count += x

    
    return analysis, buy_count# - sell_count




In [7]:
current_date = datetime.now().strftime("%Y-%m-%d")

def save_df_to_excel(df, file_name_prefix='DailyResult', sheet_name='Sheet1'):
    current_date =df.iloc[-1]['年月日'].replace('/','-')
    savedata=df[['stock_number','年月日'
                         ,'收盤價'
                         ,'Signal_Balance_uptrend_3days'
                         ,'Weighted_Signa_over_threshold'
                         ,'Signal_Balance_roll','Weighted_Signal_golden_cross','Trend']].iloc[-1]
    savedata=df.iloc[-1]
    df=savedata.to_frame().T

    if GetStockInfoByID(savedata['stock_number']).split('-')[2]!='上市':
        file_name_prefix=file_name_prefix+'2'
        
   
   
     
    # 生成帶有動態日期的文件名
    directory='log/DailyResult'
    file_name = f"{directory}/{file_name_prefix}_{current_date}.xlsx"
    file_name_last=f"{file_name_prefix}Last.xlsx"

    try:
        # 嘗試加載現有的 Excel 文件
        print(file_name)
        book = load_workbook(file_name)
        
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 加載現有數據
            writer.book = book
            if sheet_name in book.sheetnames:
                existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
                df = pd.concat([existing_df, df], ignore_index=True)
            
            # 將 DataFrame 寫入指定的 Sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            df.to_excel(file_name_last, index=False)

    except FileNotFoundError:
        # 如果文件不存在，創建新文件並寫入 DataFrame
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            df.to_excel(file_name_last, index=False)

In [8]:
def save_plt_to_html(stock_number,stock_data,fig,text_area):
    try:
        directory = f"Html"    #---本地路徑
        _filename=GetStockInfoByID(stock_number).replace('*', '')
        print(_filename)
        #檔案路徑設定
        _FilePath = f"{directory}/[{stock_number}]{_filename}.html"  #---本地路徑

        #確認資料夾是否已存在
        if not os.path.exists(directory):
            os.makedirs(directory)

        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++html處理額外新增
        #先存下html在取出，針對head 寫入
        # 需先存下html
        
        #FilePath = fr"{directory}\[{stock_number}]{_filename}.html"

        fig.write_html(_FilePath)

        # 读取现有的 HTML 文件
        with open(_FilePath, "r", encoding="utf-8") as file:
            soup = BeautifulSoup(file, "html.parser")

        # html insert 
        head = soup.find('head')
        head.append(BeautifulSoup(text_area, "html.parser"))
        #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++完成html內容部分

        ## 保存修改后的 HTML 文件 所有的股票都會先存下
        with open(_FilePath, "w", encoding="utf-8") as file:
            file.write(str(soup))

        lastdata=stock_data.iloc[-1]
        
    except Exception as error:
        print('save_plt_to_html') 
        print(error) 

In [9]:
def gen_html(stock_number,stock_data):
    
    # 計算 MACD 黃金交叉和死亡交叉
    # 黃金交叉：當前 MACD 線由下而上穿過信號線
    macd_golden_crosses = stock_data[
        (stock_data['MACD'] > stock_data['Signal_Line']) &  # 當前 MACD 線高於信號線
        (stock_data['MACD'].shift(1) <= stock_data['Signal_Line'].shift(1))  # 前一日 MACD 線低於或等於信號線
    ]

    # 死亡交叉：當前 MACD 線由上而下穿過信號線
    macd_death_crosses = stock_data[
        (stock_data['MACD'] < stock_data['Signal_Line']) &  # 當前 MACD 線低於信號線
        (stock_data['MACD'].shift(1) >= stock_data['Signal_Line'].shift(1))  # 前一日 MACD 線高於或等於信號線
    ]
    
    Weighted_Signal_golden_cross = stock_data[
        (stock_data['Weighted_Signal_roll'] > stock_data['Weighted_Signal_roll_']) &  # 當前 MACD 線低於信號線
        (stock_data['Weighted_Signal_roll'].shift(1) <= stock_data['Weighted_Signal_roll_'].shift(1))  # 前一日 MACD 線高於或等於信號線
    ]
    
    ## 計算 KD 黃金交叉和死亡交叉
    ## 黃金交叉：當前 %K 線由下而上穿過 %D 線
    #kd_golden_crosses = stock_data[
    #    (stock_data['%K'] > stock_data['%D']) &  # 當前 %K 線高於 %D 線
    #    (stock_data['%K'].shift(1) <= stock_data['%D'].shift(1))  # 前一日 %K 線低於或等於 %D 線
    #]

    # 死亡交叉：當前 %K 線由上而下穿過 %D 線
    #kd_death_crosses = stock_data[
    #    (stock_data['%K'] < stock_data['%D']) &  # 當前 %K 線低於 %D 線
    #    (stock_data['%K'].shift(1) >= stock_data['%D'].shift(1))  # 前一日 %K 線高於或等於 %D 線
    #]

    # 計算 RSI 的買入點－找出超賣區回升的點
    # 當 %K 線從 20 以上跌破 20，且 %D 線也在 20 以下時，標記為 RSI 買入點
    rsi_buy_points = stock_data[
        (stock_data['%K'] < 20) &  # 當前 %K 線低於 20
        (stock_data['%K'].shift(1) >= 20) &  # 前一日 %K 線高於或等於 20
        (stock_data['%D'] < 20)  # 當前 %D 線低於 20
    ]

    # 找出超買區回落的點
    # 當 %K 線從 80 以下上升到 80 以上，且 %D 線也在 80 以上時，標記為 RSI 賣出點
    rsi_sell_points = stock_data[
        (stock_data['%K'] > 80) &  # 當前 %K 線高於 80
        (stock_data['%K'].shift(1) <= 80) &  # 前一日 %K 線低於或等於 80
        (stock_data['%D'] > 80)  # 當前 %D 線高於 80
    ]

    # 找出適合買入的點（成交量震盪指標大於 0）
    # 當前成交量震盪指標大於 0，且前一日成交量震盪指標小於或等於 0，標記為買入點
    buy_points = stock_data[
        (stock_data['Volume_Oscillator'] > 0) &  # 當前成交量震盪指標大於 0
        (stock_data['Volume_Oscillator'].shift(1) <= 0)  # 前一日成交量震盪指標小於或等於 0
    ]

    # 找出符合加權信號閾值的買入點
    # 當加權信號大於閾值時，標記為符合加權信號條件的買入點
    buy_points_Weighted_Signal = stock_data[
        (stock_data['Weighted_Signa_over_threshold'] == True)  # 加權信號大於閾值
    ]
  
    # 創建圖表
    fig = subplots.make_subplots(rows=4, cols=1, 
                        subplot_titles=('股價與移動平均線', '成交金額與成交量震盪指標', 'KD 指標', 'MACD 指標'),
                        shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{}], [{}]],
                        row_heights=[0.6, 0.1, 0.1, 0.1]  # 调整各行的高度比例
                        )
    # 添加股票箱型圖到第一圖
    fig.add_trace(go.Candlestick(x=stock_data['年月日'],
                             open=stock_data['開盤價'],
                             high=stock_data['最高價'],
                             low=stock_data['最低價'],
                             close=stock_data['收盤價'],
                             name='箱型圖',
                             increasing_line_color='red', 
                             decreasing_line_color='green',
                             increasing_fillcolor='rgba(255,0,0,0.3)',
                             decreasing_fillcolor='rgba(0,255,0,0.3)'), row=1, col=1)
    #### 圖一 #####
    # 添加股價、短期和長期移動平均線、黃金交叉和死亡交叉到第一圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['收盤價'],                  
                             mode='lines', line_color='grey',
                             name='股價'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_short'],                  
                             mode='lines', line_color='blue', 
                             name='壓力-短期移動平均線 (MA 20)'), row=1, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MA_long'],                  
                             mode='lines', line_color='red', 
                             name='支撐-長期移動平均線 (MA 50)'), row=1, col=1)
    
    # 添加 Weighted_Signal 和 記到第一圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Weighted_Signal_roll_'],
                             mode='lines', line_color='#67ceeb',text=stock_data['Analysis'],
                             name='Weighted_Signall 20'), row=1, col=1, secondary_y=True)

    # 添加 Signal_Balance_roll 和 上升趨勢的標記到第一圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             #y=stock_data['Signal_Balance_roll'],
                             y=stock_data['Weighted_Signal_roll'],
                             mode='lines', line_color='orange',text=stock_data['Analysis'],
                            # name='短期強勢信號(10) MA/成交量/KD/MACD'), row=1, col=1, secondary_y=True)
                              name='Weighted_Signal 10'), row=1, col=1, secondary_y=True)
    
    fig.add_trace(go.Scatter(x=stock_data[stock_data['Signal_Balance_uptrend_3days']]['年月日'],
                             y=stock_data[stock_data['Signal_Balance_uptrend_3days']]['收盤價'],
                             mode='markers', marker_symbol="triangle-up", marker_color="gold", marker_size=10,text=stock_data['Analysis'],
                             name='觀察點-連續三日上升短期強勢信號'), row=1, col=1, secondary_y=False)
    
     # 添加 綜合推薦指標 標記到第一圖
    #fig.add_trace(go.Scatter(x=buy_points_Weighted_Signal['年月日'],
    #                         y=buy_points_Weighted_Signal['收盤價'],
    #                         mode='markers', marker_symbol="star", 
    #                         marker_color="red", marker_size=10,text=buy_points_Weighted_Signal['Analysis'],
    #                         name='轉強點-加權市場信號'), row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=stock_data[stock_data['MA_break']]['年月日'],
                         y=stock_data[stock_data['MA_break']]['MA_long'],
                         mode='markers', marker_symbol="star", 
                         marker_color="red", marker_size=10,text=buy_points_Weighted_Signal['Analysis'],
                         name='MA_break'), row=1, col=1, secondary_y=False)
    
    fig.add_trace(go.Scatter(x=Weighted_Signal_golden_cross['年月日'],
                         y=Weighted_Signal_golden_cross['Weighted_Signal_roll'],
                         mode='markers', marker_symbol="star", 
                         marker_color="red", marker_size=10,text=buy_points_Weighted_Signal['Analysis'],
                         name='轉強點-加權市場信號'), row=1, col=1, secondary_y=True)
    
    #### 圖二 #####

    # 添加成交金額、短期和長期移動平均線、成交量震盪指標以及買入點到第二圖
    fig.add_trace(go.Bar(x=stock_data['年月日'],
                         y=stock_data['成交金額'],
                         name='成交金額',
                         marker_color=stock_data['Bar_Color'],
                         opacity=0.5), row=2, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_MA_short'],
                             mode='lines', line_color='orange',
                             name='成交金額短期移動平均線 (MA 20)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_MA_long'],
                             mode='lines', line_color='purple',
                             name='成交金額長期移動平均線 (MA 50)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Volume_Oscillator'],
                             mode='lines', line_color='green',
                             name='成交量震盪指標 (VO)'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(x=buy_points['年月日'],
                             y=buy_points['Volume_Oscillator'],
                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='成交量增加'), row=2, col=1, secondary_y=True)

    # 添加 KD 指標和黃金交叉到第三圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['%K'],
                             mode='lines', line_color='#ebbd67',
                             name='%K(9)'), row=3, col=1)
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['%D'],
                             mode='lines', line_color='#67ceeb',
                             name='%D(3)'), row=3, col=1)
    fig.add_trace(go.Scatter(x=rsi_buy_points ['年月日'], 
                             y=rsi_buy_points ['%K'], 
                             mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=10,
                             name='超賣區回升'), row=3, col=1)
    fig.add_trace(go.Scatter(x=rsi_sell_points ['年月日'], 
                             y=rsi_sell_points ['%K'], 
                             mode='markers', marker_symbol="triangle-down", marker_color="black", marker_size=10,
                             name='超買區回落'), row=3, col=1)

    # 添加 MACD 指標、Signal Line、DIF 以及 MACD 的黃金交叉和死亡交叉到第四圖
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['MACD'],
                             mode='lines', line_color='#ebbd67',
                             name='Diff(12,26)'), row=4, col=1)
    
    fig.add_trace(go.Scatter(x=stock_data['年月日'],
                             y=stock_data['Signal_Line'],
                             mode='lines', line_color='#67ceeb',
                             name='MACD(9)'), row=4, col=1)
    
    fig.add_trace(go.Scatter(x=macd_golden_crosses['年月日'], 
                             y=macd_golden_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-up", marker_color="gold", marker_size=10,
                             name='MACD 黃金交叉'), row=4, col=1)
    fig.add_trace(go.Scatter(x=macd_death_crosses['年月日'], 
                             y=macd_death_crosses['MACD'], 
                             mode='markers', marker_symbol="triangle-down", marker_color="black", marker_size=10,
                             name='MACD 死亡交叉'), row=4, col=1)

    # 設置 x 和 y 軸
    fig.update_xaxes(title_text="日期", row=4, col=1)
 
    
    fig.update_yaxes(title_text="股價", row=1, col=1)
    fig.update_yaxes(title_text="成交金額", row=2, col=1)
    fig.update_yaxes(title_text="成交量震盪指標 (VO)", row=2, col=1, secondary_y=True, range=[stock_data['Volume_Oscillator'].min(), stock_data['Volume_Oscillator'].max()])
    fig.update_yaxes(title_text="KD 指標 (%)", row=3, col=1)
    fig.update_yaxes(title_text="MACD", row=4, col=1)
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)

    ########################
    #  想在圖片右下角備註一些指標情況
    ########################
    
    # 定义灯号和文字的位置、颜色
    transparent_color = 'gray'
    indicator_positions = [1, 2, 3, 4]
    indicator_labels = [
        'MACD 黃金交叉',
        'KD 黃金交叉',
        '觀察點-連續三日上升短期強勢信號',
        '轉強點-加權市場信號'
    ]
    indicator_colors = [
        'red' if stock_data['MACD_golden_cross'].iloc[-1] else transparent_color,
        'red' if stock_data['KD_golden_cross'].iloc[-1] else transparent_color,
        'red' if stock_data['Signal_Balance_uptrend_3days'].iloc[-1] else transparent_color,
        'red' if stock_data['Weighted_Signa_over_threshold'].iloc[-1] else transparent_color
    ]

    # 生成 HTML 内容
    html_content = ""
    for label, color in zip(indicator_labels, indicator_colors):
        html_content += f"""
        <div style="display: flex; align-items: center;">
            <div style="width: 10px; height: 10px; border-radius: 50%; background-color: {color}; margin-right: 8px;"></div>
            <span>{label}</span>
        </div><br>
    """
        
    html_iframe = f"""
    <div style="width: 100%; height: 250%; overflow: auto;">
      
        <div id="iframe-container" style="height:750px;display: none;">
            <iframe src="https://www.wantgoo.com/stock/{stock_number}" width="100%" height="100%" frameborder="0"></iframe>
        </div>
        <button onclick="toggleIframe()">顯示/隱藏 iframe</button>
        <a href="https://www.wantgoo.com/stock/{stock_number}"  target="_blank">玩股網</a>
        <a href=" https://tw.stock.yahoo.com/quote/{stock_number}.TW"  target="_blank">yahoo</a>
        <a href=" https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a={stock_number}"  target="_blank">moneydj</a>
    </div>
    <script>
        function toggleIframe() {{
            var iframeContainer = document.getElementById('iframe-container');
            if (iframeContainer.style.display === 'none') {{
                iframeContainer.style.display = 'block';
            }} else {{
                iframeContainer.style.display = 'none';
            }}
        }}
    </script>
    
    """
    # 添加 HTML 内容到图表
    html_text =html_iframe + '<br>'+ html_content+ '<br>'+ stock_data['Analysis'].iloc[-1]

    # 创建新的文本区域并使用 CSS 定位到右下角
    text_area = f'''
    <style>
        .text-area {{
            position: fixed;
            bottom: 10px;
            right: 10px;
            background-color: white;
            padding: 10px;
            border: 1px solid black;
            box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
            z-index: 1000;
            max-width: 300px;
            word-wrap: break-word;
        }}
    </style>
    <div class="text-area">
        {html_text}
    </div>
    '''

    # 顯示圖表
    #fig.show()

    #將想看的資料指標先放到excel中方便後續復盤
    #save_df_to_excel(stock_data)
    save_plt_to_html(stock_number,stock_data,fig,text_area)
   
   

In [10]:
def save_old_RunRealTimeStock(stock_data):
    _stock_data=stock_data.iloc[::-1][:1]
    to_RunRealTimeStock_excel=pd.DataFrame()
    to_RunRealTimeStock_excel['stock_number']=_stock_data['stock_number']
    to_RunRealTimeStock_excel['now_price']=_stock_data['收盤價']
    to_RunRealTimeStock_excel['change_price']=_stock_data['漲跌價差']
    to_RunRealTimeStock_excel['change_quote']= ((_stock_data['收盤價']-_stock_data['開盤價'])/_stock_data['開盤價']*100 )
    to_RunRealTimeStock_excel['change_quote']=str(format(float(to_RunRealTimeStock_excel['change_quote']), '.2f') )  +'%'
    
    _file_name = 'DailyResultWeb.xlsx'
    sheet_name='Sheet1'
    current_date =_stock_data.iloc[0]['日期'].replace("/", "-")
    log_directory='log/RunRealTimeStock'
    
    file_name = f"{log_directory}/{current_date}{_file_name}"    #---本地路徑
    df=to_RunRealTimeStock_excel

    
    try:
        # 嘗試加載現有的 Excel 文件
        book = load_workbook(file_name)
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 加載現有數據
         
            writer.book = book
            if sheet_name in book.sheetnames:
                existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
                df_new = pd.concat([existing_df, df], ignore_index=True)

            # 將 DataFrame 寫入指定的 Sheet
            df_new.to_excel(writer, sheet_name=sheet_name, index=False)
            writer.save()
    except FileNotFoundError:
        # 如果文件不存在，創建新文件並寫入 DataFrame
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)

            

from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

start_month='2023-01-01'
end_time_list=['113/11/11'
               # '113/11/08','113/11/07','113/11/06','113/11/05', '113/11/04',
               # '113/11/01','113/10/30','113/10/29', '113/10/28',
               ]

stock_number_list= _baseInfo.dropna()['公司代號'][::-1]

#stock_number_list= _baseInfo[_baseInfo['Type']=='上櫃'].dropna()['公司代號']
#stock_number_list=['8923'] 

# 定义保存 Excel 文件的函数
def save_process_data(stock_number,stock_data,end_time):
    #以下多執行處理
    try:
        #stock_data=data_process(RowData_df_craw_stock[RowData_df_craw_stock['日期']<=end_time])
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
      
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        
        save_old_RunRealTimeStock(stock_data)
        save_df_to_excel(stock_data)
        
        if max(end_time_list)==end_time :
            gen_html(stock_number,stock_data)
            
    except Exception as error:
        print('save_process_data') 
        print(error) 

# 定义主要爬虫和处理函数
def process_stock(stock_number):
    try:
        # 依次获取数据（爬虫部分）
        
        RowData_df_craw_stock, His_Stock, isSuccess = craw_stock(stock_number, start_month)

        # 保存文件的线程池
        with ThreadPoolExecutor(max_workers=5) as executor:
            # 提交每个 end_time 的处理任务
            futures = []
            for end_time in end_time_list[::-1]:
                stock_data = data_process(RowData_df_craw_stock[RowData_df_craw_stock['日期'] <= end_time])
                # 提交保存任务
                futures.append(executor.submit(save_process_data,stock_number, stock_data,end_time))
                
            # 等待所有保存任务完成
            for future in as_completed(futures):
                future.result()  # 捕获异常并确认任务完成

    except Exception as error:
        print(f'Error processing stock {stock_number}: {error}')

# 主循环依次执行每个股票的爬虫处理
for stock_number in stock_number_list:
    print(stock_number)
    process_stock(stock_number)
print("All stocks processed.")